In [ ]:
%matplotlib inline
import os
# import keras
# from keras import backend as K
# from keras.preprocessing import image
# import tensorflow as tf
from matplotlib import pyplot as plt
import cv2
import numpy as np


def calc_bandwidth(lambd, sigma):
    r = np.pi*sigma/lambd
    c = np.sqrt(np.log(2)/2)
    return np.log2((r + c)/(r - c))

def calc_sigma(lambd, bandwidth):
    p = 2**bandwidth
    c = np.sqrt(np.log(2)/2)
    return lambd * c / np.pi  * (p + 1) / (p - 1)

def calc_lambda(sigma, bandwidth):
    p = 2**bandwidth
    c = np.sqrt(np.log(2)/2)
    return sigma * np.pi / c  * (p - 1) / (p + 1)

# Set parameters
project_root = os.path.realpath('.')  # os.pardir
fig_dir = os.path.join(project_root, "results", "figs")
if not os.path.isdir(fig_dir):
    os.makedirs(fig_dir)

In [ ]:
calc_sigma(5, 1)

# Plot filters

In [ ]:
# image_path = "/work/data/Lenna.png"
image_path = "/work/data/example_aeroplane_s_000021.png"
# image_path = "/work/data/4.2.03.tiff"  # Mandrill
# image_path = "/work/data/4.2.07.tiff"  # Peppers
# img = image.load_img(image_path)
# img = plt.imread(image_path)
img = cv2.imread(image_path)
# img = np.asarray(img, dtype=np.uint8)

print(np.amin(img), np.amax(img))
print(img.shape)

img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
img = np.copy(img.astype('float32')) / 255
# img = img.astype('float32')
print(type(img))
img = K.expand_dims(img, 0)
img = K.expand_dims(img, -1)
print(img.shape)
print(np.amin(img), np.amax(img))

lambdas = [3, 4, 5, 6, 7, 8]  # 3 <= lambd <= W/2
# sigmas = [1, 2, 3] # 4]  
# bandwidths = np.linspace(0.4, 2.6, num=3)  # ~1.5 <= bw <= ~3
bandwidths = np.linspace(1, 1.8, num=3)

lambdas = [5]
bandwidths = [1.4]

n_thetas = 8
n_psis = 4  # 1, 2, 4
n_gammas = 2

thetas = np.linspace(0, np.pi, n_thetas, endpoint=False)
psis = np.linspace(0, 2*np.pi, n_psis, endpoint=False)
gammas = np.linspace(1, 0, n_gammas, endpoint=False)

convolve = True
size = (31, 31)
# gabor = {'sigmas': [sigma],
#          'lambdas': [lambd],
#          'thetas': thetas,        
#          'psis': psis,
#          'gammas': gammas,
#          'ksize': (31, 31),
#         }

ncols = len(thetas)
# nrows = int(np.ceil(len(gammas)*len(psis)))
nrows = len(psis)

fontsize = 16  # 20
space = 0.15
width = 12
print(f"Total Gabor filters: {ncols*nrows}")

img_scale = 6

if convolve:
    nrows *= 2

i = 0

for gm, gamma in enumerate(gammas):
    i = 0
    fig, axes = plt.subplots(nrows=nrows, ncols=ncols, sharex='row', sharey='row', figsize=(width, width*nrows/ncols))
    for bw in bandwidths:
        for lm, lambd in enumerate(lambdas):
            sigma = calc_sigma(lambd, bw)
    #         lambd = calc_lambda(sigma, bw)
#             fig, axes = plt.subplots(nrows=nrows, ncols=ncols, sharex='row', sharey='row', figsize=(width, width*nrows/ncols))
#         for gm, gamma in enumerate(gammas):
            for ps, psi in enumerate(psis):
                for th, theta in enumerate(thetas):

                    params = {'ksize': size, 'sigma': sigma, 'theta': theta, 'lambd': lambd, 'gamma': gamma, 'psi': psi}
                    gf = cv2.getGaborKernel(**params, ktype=cv2.CV_32F)

                    row, col = (i//ncols), i%ncols
                    if convolve:
                        row *= 2
                    axes[row, col].imshow(gf, cmap='gray', vmin=-1, vmax=1)
                    axes[row, col].set_xticks([])
                    axes[row, col].set_yticks([])
                    # print(np.amin(gf), np.amax(gf))
#                     simplify(th*np.pi/n_thetas)
                    if i//ncols == 0:
#                         axes[row, col].set_title(r"$\theta = {:.3}\pi$".format(theta/np.pi))

                        if th == 0:
                            axes[row, col].set_title(r"$\theta = 0$", fontsize=fontsize)
                        else:
                            axes[row, col].set_title(r"$\theta = \frac{{{}}}{{{}}}\pi$".format(th, n_thetas), fontsize=fontsize)
                    if i%ncols == 0:
#                         axes[row, col].set_ylabel(r"$\psi = {:.3}\pi, \gamma = {}$".format(psi/np.pi, gamma))  #lambd, sigma))
                        if ps == 0:
                            axes[row, col].set_ylabel(r"$\psi = 0, \gamma = {}$".format(gamma), fontsize=fontsize)
                        else:
                            axes[row, col].set_ylabel(r"$\psi = \frac{{{}}}{{{}}}\pi, \gamma = {}$".format(ps, n_psis, gamma), fontsize=fontsize)

                    if convolve:
                        gf = K.expand_dims(gf, -1)
                        gf = K.expand_dims(gf, -1)
                        # https://stackoverflow.com/questions/34619177/what-does-tf-nn-conv2d-do-in-tensorflow
                        # K.conv2d(image.img_to_array(img), gf)
                        fimg = K.conv2d(img, gf, padding='same')
                        fimg = tf.Session().run(fimg[0,:,:,0])
                        axes[row+1, col].imshow(fimg, cmap='gray', vmin=-img_scale, vmax=img_scale)
                        # axes[row+1, col].imshow(fimg[0,:,:,0].eval(), cmap='gray')
                        axes[row+1, col].set_xticks([])
                        axes[row+1, col].set_yticks([])
                        print(np.amin(fimg), np.amax(fimg))
                    i += 1
    plt.tight_layout()
    plt.subplots_adjust(wspace=space, hspace=space)
    plt.savefig(os.path.join(fig_dir, f"gabor_kernels_{gm}.png"), bbox_inches="tight")  # , additional_artists=[lgd])

## Plot variations of filters

In [ ]:
image_path = "/work/data/example_aeroplane_s_000021.png"
img = cv2.imread(image_path)
# img = np.asarray(img, dtype=np.uint8)
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
img = np.copy(img.astype('float32')) / 255
# img = img.astype('float32')
img = K.expand_dims(img, 0)
img = K.expand_dims(img, -1)

# sigmas = [0.2, 0.4, 0.6, 0.8, 1, 1.2, 1.4, 1.6, 1.8, 2, 2.2, 2.4, 2.6, 2.8, 3, 4, 5]
sigmas = [1, 1.2, 1.4, 1.6, 1.8, 2, 2.2, 2.4, 2.6, 2.8, 3, 4, 5]
sigmas = range(1, 6)
bandwidths = np.linspace(1, 1.8, num=5)
theta = np.pi/4
gamma = 0.5
psis = [0, np.pi/2]

convolve = False
size = (31, 31)

nrows = len(sigmas)
ncols = len(bandwidths)

fontsize = 16  # 20
space = 0.15
width = 12
print(f"Total Gabor filters: {ncols*nrows}")

img_scale = 6

if convolve:
    nrows *= 2

for ps, psi in enumerate(psis):
    i = 0
    fig, axes = plt.subplots(nrows=nrows, ncols=ncols, sharex='row', sharey='row', figsize=(width, width*nrows/ncols))
    for sg, sigma in enumerate(sigmas):
        for bw in bandwidths:
            lambd = calc_lambda(sigma, bw)
    #     for lm, lambd in enumerate(lambdas):
    #         sigma = calc_sigma(lambd, bw)

            params = {'ksize': size, 'sigma': sigma, 'theta': theta, 'lambd': lambd, 'gamma': gamma, 'psi': psi}
            gf = cv2.getGaborKernel(**params, ktype=cv2.CV_32F)

            row, col = (i//ncols), i%ncols
            if convolve:
                row *= 2
            axes[row, col].imshow(gf, cmap='gray', vmin=-1, vmax=1)
            axes[row, col].set_xticks([])
            axes[row, col].set_yticks([])

            if i//ncols == 0:
                axes[row, col].set_title(r"$b = {}$".format(bw), fontsize=fontsize)
            if i%ncols == 0:
                axes[row, col].set_ylabel(r"$\sigma = {}$".format(sigma), fontsize=fontsize)

            if convolve:
                gf = K.expand_dims(gf, -1)
                gf = K.expand_dims(gf, -1)
                # https://stackoverflow.com/questions/34619177/what-does-tf-nn-conv2d-do-in-tensorflow
                # K.conv2d(image.img_to_array(img), gf)
                fimg = K.conv2d(img, gf, padding='same')
                fimg = tf.Session().run(fimg[0,:,:,0])
                axes[row+1, col].imshow(fimg, cmap='gray', vmin=-img_scale, vmax=img_scale)
                # axes[row+1, col].imshow(fimg[0,:,:,0].eval(), cmap='gray')
                axes[row+1, col].set_xticks([])
                axes[row+1, col].set_yticks([])
    #             print(np.amin(fimg), np.amax(fimg))
            i += 1
    plt.tight_layout()
    plt.subplots_adjust(wspace=space, hspace=space)
    plt.savefig(os.path.join(fig_dir, f"gabor_kernels_variation_{ps}.png"), bbox_inches="tight")  # , additional_artists=[lgd])

# Plot Results

In [ ]:
# Load accuracy scores and plot
%matplotlib inline
import json
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

model = 'gabor'
data_set = 'pixel'
stimulus_set = 'jitter'  # 'static'  # 'jitter'
noise_types = ['Original', 'Salt-and-pepper', 'Additive', 'Single-pixel']  # 'Original'
test_conditions = ['Same', 'Diff', 'NoPix']

start_trial = 1
num_trials = 5
sigmas = [0.2, 0.4, 0.6, 0.8, 1, 1.2, 1.4, 1.5, 1.6, 1.8, 2, 2.2, 2.4, 2.5, 2.6, 2.8, 3, 3.5, 4, 4.5, 5]
bandwidths = np.linspace(1, 1.8, num=5)

epochs = 20
save_loss = 0
results_dir = os.path.join(project_root, 'results', model, data_set, stimulus_set)

rows = []
test_rows = []

# for stimulus_set in stimulus_sets:
for trial in range(1, 1+num_trials):
    for noise_type in noise_types:
#         for lambd in lambdas:
        for sigma in sigmas:
            for bandwidth in bandwidths:
#                 sigma = calc_sigma(lambd, bandwidth)
                lambd = calc_lambda(sigma, bandwidth)

                trial_label = f"{trial}_sigma={float(sigma):.2}_lambd={float(lambd):.2}"
                model_name = f"{noise_type}_{trial_label}"
                # print(model_name)

                acc_scores = np.load(os.path.join(results_dir, f'{model_name}_ACC.npy'))
                valacc_scores = np.load(os.path.join(results_dir, f'{model_name}_VALACC.npy'))
                if save_loss:
                    loss = np.load(os.path.join(results_dir, f'{model_name}_LOSS.npy'))
                    valloss = np.load(os.path.join(results_dir, f'{model_name}_VALLOSS.npy'))
                else:
                    loss = np.zeros(epochs)
                    valloss = np.zeros(epochs)

                with open(os.path.join(results_dir, f'{model_name}_CONDVALACC.json'), "r") as jf:
                    cond_acc = json.load(jf)
                if save_loss:
                    with open(os.path.join(results_dir, f'{model_name}_CONDVALLOSS.json'), "r") as jf:
                        cond_loss = json.load(jf)
                else:
                    cond_loss = {condition: 0 for condition in test_conditions}

                for condition in test_conditions:
                    test_rows.append({'Trial': trial, 'Noise': noise_type, 'Condition': condition, 
                                      'Scale': sigma, 'Wavelength': lambd, 'Bandwidth': bandwidth,
                                      'Loss': cond_loss[condition], 'Accuracy': cond_acc[condition]})
                for epoch in range(epochs):
                    rows.append({'Trial': trial, 'Noise': noise_type, 'Epoch': epoch+1, 'Evaluation': 'Testing', 
                                 'Scale': sigma, 'Wavelength': lambd, 'Bandwidth': bandwidth, 
                                 'Loss': valloss[epoch], 'Accuracy': valacc_scores[epoch]})

                    rows.append({'Trial': trial, 'Noise': noise_type, 'Epoch': epoch+1, 'Evaluation': 'Training', 
                                 'Scale': sigma, 'Wavelength': lambd, 'Bandwidth': bandwidth, 
                                 'Loss': loss[epoch], 'Accuracy': acc_scores[epoch]})

scores = pd.DataFrame(rows, columns=['Trial', 'Noise', 'Epoch', 'Evaluation', 'Scale', 'Wavelength', 'Bandwidth', 'Loss', 'Accuracy'])
test_scores = pd.DataFrame(test_rows, columns=['Trial', 'Noise', 'Condition', 'Scale', 'Wavelength', 'Bandwidth', 'Loss', 'Accuracy'])

symbols = {'Orientation': '$\theta$',
           'Phase': '$\psi$',
           'Aspect': '$\gamma$',
           'Scale': '$\sigma$',
           'Wavelength': '$\lambda$',
           'Bandwidth': '$b$'}

units = {'Orientation': 'radians',
         'Phase': 'radians',
         'Aspect': '',
         'Scale': '',  # Check
         'Wavelength': 'pixels',  # 'pixels/cycle'
         'Bandwidth': 'octaves'}  # Check

if not save_loss:
    scores.drop(columns='Loss', inplace=True)
    test_scores.drop(columns='Loss', inplace=True)
# scores.rename(columns={'Noise': 'Mask'}, inplace=True)
scores.loc[:, 'Accuracy'] *= 100  # Convert to percentage
# test_scores.rename(columns={'Noise': 'Mask'}, inplace=True)
test_scores.loc[:, 'Accuracy'] *= 100  # Convert to percentage

In [ ]:
# display_lam = 5
display_bw = None  # 1.4
display_sig = None  # 1.8

# subfig_size = (7, 6)
sns.set_context('talk')  # talk
# sns.set(context='talk', style="white", font_scale=1.45, font='serif',
#         rc={"font.family": "serif", "font.serif": ["Times", "Palatino", "serif"]})
# sns.set(style="white")
# sns.set(font='serif')
sns.set(font_scale=2.) # 2  # 2.5
sns.set_style("whitegrid",
              {"font.family": "serif",
               "font.serif": ["Times", "Palatino", "serif"]})
palette = plt.rcParams["axes.prop_cycle"].by_key()["color"]

orig_scores = test_scores.query('Noise == "Original" and Condition == "Same"')
noise_scores = test_scores.drop(test_scores.query('Noise == "Original"').index)
new_scores = [orig_scores.replace("Original", noise).replace("Same", "None") for noise in noise_types[1:]]
recomb_scores = pd.concat([noise_scores, *new_scores])

In [ ]:
aspect = 1.  # 1.3
height = 3.7
# width = 5.5
# height = width / aspect

plot_scores = recomb_scores.query('Scale >= 1')
g = sns.relplot(x='Scale', y='Accuracy', col='Bandwidth', row='Noise', hue='Condition', 
                kind='line', data=plot_scores, palette=[*palette[:3], palette[7]], aspect=aspect, height=height) #3.7)  #5.2)
g.set(xlim=(1, 5))
g.set(ylim=(0, 100))
g.set_xlabels(r'$\sigma$')  # [pixels]
# g.set(xticks=sigmas)
g.set(xticks=list(range(1, 6)))
# g.set_titles(template=r'$b$ = {row_name} | {col_name}')  # {col_var} = 
g.set_titles(template=r'$b$ = {col_name}')
for ri, row in enumerate(g.axes):
    for ci, ax in enumerate(row):
        ax.axhline(y=10, linestyle='--', linewidth=3, color='#e74c3c')  # e74c3c
        if display_sig:
            ax.axvline(x=display_sig, linestyle=':', linewidth=3, color=palette[8])
        if ci == 0:
            ax.set_ylabel(f'Accuracy\n({noise_types[ri+1]})')

# g.set(xscale='log')
# plt.tight_layout()
g.savefig(os.path.join(fig_dir, f"relplot_sigma_acc_bw.png"), bbox_inches="tight")#, additional_artists=[lgd])

In [ ]:
plot_scores = recomb_scores.query('Scale in [1, 1.5, 2, 2.5, 3]')  # [1, 1.4, 1.8, 2.2, 2.6, 3] # .query('Scale >= 1')
# plot_scores = recomb_scores.query('1 <= Scale <= 2.4') # .query('Scale >= 1')
g = sns.relplot(x='Bandwidth', y='Accuracy', col='Scale', row='Noise', hue='Condition', 
                kind='line', data=plot_scores, palette=[*palette[:3], palette[7]], aspect=aspect, height=height)

g.set(ylim=(0, 100))
g.set(xlim=(1, 1.8))
g.set(xticks=bandwidths)
g.set_xlabels(r'$b$ [octaves]')
# g.set_titles(template=r'$\sigma$ = {row_name} | {col_name}')  # {col_var} =
g.set_titles(template=r'$\sigma$ = {col_name}')


for ri, row in enumerate(g.axes):
    for ci, ax in enumerate(row):
        ax.axhline(y=10, linestyle='--', linewidth=3, color='#e74c3c')
        if display_bw:
            ax.axvline(x=display_bw, linestyle=':', linewidth=3, color=palette[8])
        if ci == 0:
            ax.set_ylabel(f'Accuracy\n({noise_types[ri+1]})')

# plt.tight_layout()
g.savefig(os.path.join(fig_dir, f"relplot_bw_acc_sigma.png"), bbox_inches="tight")

In [ ]:
calc_lambda(1.5, 1)

In [ ]:
subfig_size = (7, 6)
sns.set(font_scale=2.) # 2  # 2.5
sns.set_style("white",
              {"font.family": "serif",
               "font.serif": ["Times", "Palatino", "serif"]})

display_scale = 2.5

query_string = 'Bandwidth in [1, 1.4, 1.8] and Scale == "{}"'.format(display_scale)
display_data = noise_scores.query(query_string)
for noise in noise_types[1:]:
    fig, ax = plt.subplots(figsize=subfig_size)
    bp = sns.barplot(x="Condition", y="Accuracy", hue='Bandwidth', 
                     data=display_data.query(f"Noise == '{noise}'"), 
                     capsize=0.2, color='grey', edgecolor=".1", ax=ax)

    ax.set_ylim(0, 100)
    ax.hlines(10, -0.5, 2.5, linestyle='--', linewidth=3, colors='#e74c3c')
    mean_scores = orig_scores.query(query_string).mean()
    ax.hlines(mean_scores['Accuracy'], -0.5, 2.5, linestyle=':', linewidth=3, colors=palette[7])
#     ax.set_xlabel('')
#     ax.set_title(noise, fontsize='x-large')
    ax.set_xlabel(noise, fontsize='x-large')
#     ax.set_ylabel(ax.get_ylabel()+f'\n{noise}', fontsize='x-large')
    lgd = bp.legend(loc=1, framealpha=0.5, fontsize=18, ncol=1, frameon=True, fancybox=True, title_fontsize=18, title="Bandwidth"), 
#                     handles=handles[1:], labels=labels[1:], bbox_to_anchor=(1.0, 0.65))
    plt.tight_layout()
    plt.savefig(os.path.join(fig_dir, f"UKCN_{noise}_display_bar.png"), bbox_inches="tight", additional_artists=[lgd])

In [ ]:
!touch ../results/gabor/pixel/jitter/zzz